Beat-n game with Nash-env derived from 15 pt limit and augmented with higher-cost sub-Nash Spuds.

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [57]:
const ability_none = 999
const ability_your_L_their_S = 500
const ability_your_P_their_R = 550
const ability_your_R_their_S = 600

ability_name = Dict(
    ability_your_L_their_S => "yourLtheirS",
    ability_your_P_their_R => "yourPtheirR",
    ability_your_R_their_S => "yourRtheirS",
)

function fx_your_L_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, b.s, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, b.r, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, b.s, a.s, a.a1)
    return [a, b]
end



ability_fx = Dict(
    ability_your_L_their_S => fx_your_L_their_S,
    ability_your_P_their_R => fx_your_P_their_R,
    ability_your_R_their_S => fx_your_R_their_S,
)

Dict{Int64, Function} with 3 entries:
  500 => fx_your_L_their_S
  550 => fx_your_P_their_R
  600 => fx_your_R_their_S

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 15 and filter nondominated

In [10]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [11]:
lib15 = get_library(15)
lib15;

In [12]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [29]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [30]:
@time lib=filter_nondominated2(lib15)
# for ff in lib
#     print(" ")
#     print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
# end

 35.326648 seconds (71.82 M allocations: 125.241 GiB, 34.12% gc time, 0.42% compilation time)


1396-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 2, 9, 999)
 Spud("", 1, 1, 1, 1, 3, 8, 999)
 Spud("", 1, 1, 1, 1, 4, 7, 999)
 Spud("", 1, 1, 1, 1, 5, 6, 999)
 Spud("", 1, 1, 1, 1, 6, 5, 999)
 Spud("", 1, 1, 1, 1, 7, 4, 999)
 Spud("", 1, 1, 1, 1, 8, 3, 999)
 Spud("", 1, 1, 1, 1, 9, 2, 999)
 Spud("", 1, 1, 1, 2, 1, 9, 999)
 Spud("", 1, 1, 1, 2, 2, 8, 999)
 Spud("", 1, 1, 1, 2, 3, 7, 999)
 Spud("", 1, 1, 1, 2, 4, 6, 999)
 Spud("", 1, 1, 1, 2, 5, 5, 999)
 ⋮
 Spud("", 7, 1, 3, 1, 1, 2, 999)
 Spud("", 7, 1, 3, 1, 2, 1, 999)
 Spud("", 7, 1, 3, 2, 1, 1, 999)
 Spud("", 7, 1, 4, 1, 1, 1, 999)
 Spud("", 7, 2, 2, 1, 1, 2, 999)
 Spud("", 7, 2, 2, 1, 2, 1, 999)
 Spud("", 7, 2, 2, 2, 1, 1, 999)
 Spud("", 8, 1, 2, 1, 1, 2, 999)
 Spud("", 8, 1, 2, 1, 2, 1, 999)
 Spud("", 8, 1, 2, 2, 1, 1, 999)
 Spud("", 8, 1, 3, 1, 1, 1, 999)
 Spud("", 9, 1, 2, 1, 1, 1, 999)

In [42]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

i2s = Dict(spud2int(ff) => ff for ff in lib)

Dict{Int64, Spud} with 1396 entries:
  226212 => Spud("", 2, 2, 6, 2, 1, 2, 999)
  622311 => Spud("", 6, 2, 2, 3, 1, 1, 999)
  124242 => Spud("", 1, 2, 4, 2, 4, 2, 999)
  413142 => Spud("", 4, 1, 3, 1, 4, 2, 999)
  124314 => Spud("", 1, 2, 4, 3, 1, 4, 999)
  413322 => Spud("", 4, 1, 3, 3, 2, 2, 999)
  132135 => Spud("", 1, 3, 2, 1, 3, 5, 999)
  413232 => Spud("", 4, 1, 3, 2, 3, 2, 999)
  262311 => Spud("", 2, 6, 2, 3, 1, 1, 999)
  423213 => Spud("", 4, 2, 3, 2, 1, 3, 999)
  532212 => Spud("", 5, 3, 2, 2, 1, 2, 999)
  122343 => Spud("", 1, 2, 2, 3, 4, 3, 999)
  111624 => Spud("", 1, 1, 1, 6, 2, 4, 999)
  324114 => Spud("", 3, 2, 4, 1, 1, 4, 999)
  223413 => Spud("", 2, 2, 3, 4, 1, 3, 999)
  112641 => Spud("", 1, 1, 2, 6, 4, 1, 999)
  124413 => Spud("", 1, 2, 4, 4, 1, 3, 999)
  243123 => Spud("", 2, 4, 3, 1, 2, 3, 999)
  191211 => Spud("", 1, 9, 1, 2, 1, 1, 999)
  126321 => Spud("", 1, 2, 6, 3, 2, 1, 999)
  522123 => Spud("", 5, 2, 2, 1, 2, 3, 999)
  116421 => Spud("", 1, 1, 6, 4, 2, 1, 

In [31]:
payoffs = get_payoffs(lib)

1396×1396 Matrix{Int64}:
 0  -1  -1  -1  -1  -1  -1  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   0  -1  -1  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   0  -1  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   0  -1  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   0  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   1 

In [32]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));

In [33]:
s2ind[lib[5]]

5

In [34]:
using JuMP
using HiGHS

In [35]:
vector_model = Model(HiGHS.Optimizer)
n_nash = length(lib)
sumv = zeros(Int64, (1, 2 * n_nash + 1))
sumv[1:n_nash] .= 1
A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
A = vcat(A, sumv);
b = zeros(Int64, n_nash + 1);
b[1:n_nash] .= -1
b[n_nash+1] = 1
transpose(b);
c = zeros(Int64, 2 * n_nash + 1);
c[2 * n_nash + 1] = 1;
@variable(vector_model, x[1:(2*n_nash+1)] >= 0);
@constraint(vector_model, A * x .== b);
@objective(vector_model, Min, c' * x)
@time optimize!(vector_model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1397 rows, 2793 cols, 1951608 nonzeros
1397 rows, 2793 cols, 1951608 nonzeros
Presolve : Reductions: rows 1397(-0); columns 2793(-0); elements 1951608(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1397(1397) 0s
       2920     1.0000000000e+00 Pr: 0(0) 3s
       2920     1.0000000000e+00 Pr: 0(0) 3s
Model   status      : Optimal
Simplex   iterations: 2920
Objective value     :  1.0000000000e+00
HiGHS run time      :          3.35
  3.652368 seconds (237.19 k allocations: 98.475 MiB, 4.43% compilation time: 100% of which was recompilation)


In [36]:
w_lp = [value(x[i]) for i in 1:n_nash]
sum(w_lp)

0.9999999999999993

In [37]:
w_lp = w_lp./sum(w_lp);
maximum([sum(payoffs[i, :] .* w_lp) for i in 1:n_nash])

2.886579864025407e-15

In [38]:
new_env = lib[w_lp .> 0]
w_new = w_lp[w_lp .> 0];

In [39]:

for i in sortperm(-w_new)
    print(spud2int(new_env[i]))
    print(" ")
    println(string(w_new[i])[1:6])
end

615111 0.0691
417111 0.0688
516111 0.0680
714111 0.0626
813111 0.0583
318111 0.0572
219111 0.0469
131154 0.0244
121614 0.0238
171123 0.0179
161232 0.0170
171132 0.0167
115413 0.0165
151431 0.0160
151341 0.0155
141252 0.0152
151143 0.0151
151521 0.0125
171411 0.0124
161313 0.0124
161331 0.0122
141441 0.0120
131613 0.0118
161421 0.0116
141513 0.0113
181221 0.0109
151332 0.0107
161412 0.0106
151242 0.0105
141612 0.0103
614112 0.0095
131622 0.0091
111165 0.0090
121425 0.0090
121245 0.0088
171321 0.0087
131541 0.0086
171213 0.0086
141153 0.0085
115134 0.0080
161322 0.0080
151512 0.0074
171312 0.0071
141423 0.0070
216123 0.0066
151224 0.0065
151314 0.0064
141414 0.0062
121155 0.0058
115224 0.0055
116133 0.0053
161133 0.0051
141324 0.0050
713112 0.0049
113235 0.0048
131244 0.0047
141531 0.0043
181311 0.0040
214134 0.0038
121335 0.0038
116223 0.0038
131424 0.0037
141522 0.0037
111255 0.0031
131532 0.0029
317112 0.0023
515112 0.0022
161142 0.0022
181122 0.0021
161223 0.0018
141351 0.0017
171231

In [40]:
function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

function stat_dominates(a::Spud, b::Spud)::Bool
    if minimum(spud_to_vec(a) .- spud_to_vec(b))==0
        return true
    else
        return false
    end
end

function stat_dominates_any(a::Spud, bs::Array{Spud})::Bool
    for b in bs
        if stat_dominates(a, b)
            return true
        end
    end
    return false
end

stat_dominates_any (generic function with 1 method)

## Combo breaker development

In [56]:
as = [i2s[i] for i in [416112, 141333, 113424]]
println(maximum([eval_battle_list(ff, as) for ff in lib]))
b = Spud("breaker",5,4,1,1,1,3,ability_your_L_their_S)
println([eval_battle(b, ff) for ff in as])
eval_battle_list2(b, new_env, w_new)

1
[1, 1, 1]


-0.03985525090380973

In [58]:
as = [i2s[i] for i in [417111,141342,113523]]
println(maximum([eval_battle_list(ff, as) for ff in lib]))
b = Spud("breaker",2,4,4,1,1,2,ability_your_P_their_R)
println([eval_battle(b, ff) for ff in as])
eval_battle_list2(b, new_env, w_new)

1
[1, 1, 1]


-0.08205185215389163

In [60]:
as = [i2s[i] for i in [416112,141234,113424]]
println(maximum([eval_battle_list(ff, as) for ff in lib]))
b = Spud("breaker",2,4,4,1,1,2,ability_your_R_their_S)
println([eval_battle(b, ff) for ff in as])
eval_battle_list2(b, new_env, w_new)

1
[-1, -1, 1]


-0.15145981083349505